# Multidimensional vector

We will expand the class created in chapter 11 to be more generic. By doing this, we will be able to explore more special methods applied to sequences in python.



In [1]:
# First version
# Compatible (as much as possible) with Vector2d class from chapter 11

from array import array
import reprlib
import math

class Vector:
    typecode = 'd'

    def __init__(self, components) -> None:
        self._components = array(self.typecode, components)

    def __iter__(self):
        return iter(self._components)

    def __repr__(self) -> str:
        components = reprlib.repr(self._components) # Shorten the array representation
        components = components[components.find('['):-1] # Remove array([d' and last )
        return f'Vector({components})'
    
    def __str__(self) -> str:
        return str(tuple(self)) # Why print the whole Vector?

    def __bytes__(self):
        return (bytes([ord(self.typecode)]) + bytes(self._components))

    def __eq__(self, other: object) -> bool:
        return tuple(self) == tuple(other)

    def __abs__(self) -> float:
        return math.hypot(*self) # math.hypot support n-dimensional input
    
    def __bool__(self) -> bool:
        return bool(abs(self))

    @classmethod
    def frombytes(cls, octets):
        typecode = chr(octets[0])
        memv = memoryview(octets[1:]).cast(typecode)
        return cls(memv)
    
Vector([3.1, 4.2]) # Vector([3.1, 4.2])
Vector((3, 4, 5)) # Vector([3.0, 4.0, 5.0])
Vector(range(10)) # Vector([0.0, 1.0, 2.0, 3.0, 4.0, ...])

# Duck typing and protocols

In the OOP context, a protocol is a informal interface, defined in the documentation rather than in the code itself. For example, a class that implements the sequence protocol is any class that implements the `__len__` and `__getitem__` dunder functions. Remember the `FrenchDeck` class implemented in chapter 1. This class can be used anywhere that expects a sequence. Because it does not matter if a class is derived from another to be able to be used, we can say that `FrenchDeck` is a sequence because it behaves like a sequence. This concept is called duck typing.

Because protocols are informal and not obligatory, sometimes we can implement just a portion of the protocol that we are interested. For example, just the `__getitem__` function is need to support iteration, if we need just that, we do not need to implement `__len__`

There's a difference between a protocol concept, what we have just seen, and the `typing.Protocol` (seen in chapter 8.5.10): the latter is called static protocol (PEP 554), because they are a way to formalize the protocol concept. The difference is that all functions defined in a static protocol must be implemented.

In [11]:
# Second version
# Implement the sequence protocol

from array import array
import operator
import reprlib
import math

class Vector:
    typecode = 'd'

    def __init__(self, components) -> None:
        self._components = array(self.typecode, components)

    def __iter__(self):
        return iter(self._components)

    def __repr__(self) -> str:
        components = reprlib.repr(self._components) # Shorten the array representation
        components = components[components.find('['):-1] # Remove array([d' and last )
        return f'Vector({components})'
    
    def __str__(self) -> str:
        return str(tuple(self)) # Why print the whole Vector?

    def __bytes__(self):
        return (bytes([ord(self.typecode)]) + bytes(self._components))

    def __eq__(self, other: object) -> bool:
        return tuple(self) == tuple(other)

    def __abs__(self) -> float:
        return math.hypot(*self) # math.hypot support n-dimensional input
    
    def __bool__(self) -> bool:
        return bool(abs(self))

    def __len__(self) -> int:
        return len(self._components)
    
    def __getitem__incorrect(self, index): # because the v[a:b] produces an array
        return self._components[index]

    def __getitem__(self, key):
        if isinstance(key, slice):
            cls = type(self) 
            return cls(self._components[key]) # Calls Vector constructor passing an array. Question: this is only possible because the array is iterable?
        index = operator.index(key) # Better than int(key)
        return self._components[index]

    @classmethod
    def frombytes(cls, octets):
        typecode = chr(octets[0])
        memv = memoryview(octets[1:]).cast(typecode)
        return cls(memv)
    
v1 = Vector([3, 4, 5])
len(v1) # 3
v1[0], v1[-1] # (3.0, 5.0)
v7 = Vector(range(7))
v7[1:4] # array('d', [1.0, 2.0, 3.0]) - using __getitem__incorrect
v7[1:4] # Vector([1.0, 2.0, 3.0])
v7[-1:] # Vector([6.0])
# v7[1,2] # Error: Tuple not supported for slicing (that's what we need)
# v7[3.14] # Error: float cannot be used as index (that's why we use operator.index())

Vector([6.0])

## How the slicing works in python

```python
class MySeq:
    def __getitem__(self, index):
        return index

s = MySeq()
s[1] # 1
s[1:4] # slice(1, 4, None)
s[1:4:2] # slice(1, 4, 2)
s[1:4:2, 9] # (slice(1, 4, 2), 9)
s[1:4:2, 7:9] # (slice(1, 4, 2), slice(7, 9, None))
```

Question: The builtin type `slice` has the function `indices` that I did not understand how it works.

In [ ]:
# Third version
# Implement dynamic access to Vector items
# Enables us to use v.x, v.y, v.z, v.t to access the four first dimentions of our vector

from array import array
import operator
import reprlib
import math
from typing import Any

class Vector:
    typecode = 'd'

    def __init__(self, components) -> None:
        self._components = array(self.typecode, components)

    def __iter__(self):
        return iter(self._components)

    def __repr__(self) -> str:
        components = reprlib.repr(self._components) # Shorten the array representation
        components = components[components.find('['):-1] # Remove array([d' and last )
        return f'Vector({components})'
    
    def __str__(self) -> str:
        return str(tuple(self)) # Why print the whole Vector?

    def __bytes__(self):
        return (bytes([ord(self.typecode)]) + bytes(self._components))

    def __eq__(self, other: object) -> bool:
        return tuple(self) == tuple(other)

    def __abs__(self) -> float:
        return math.hypot(*self) # math.hypot support n-dimensional input
    
    def __bool__(self) -> bool:
        return bool(abs(self))

    def __len__(self) -> int:
        return len(self._components)
    
    def __getitem__(self, key):
        if isinstance(key, slice):
            cls = type(self) 
            return cls(self._components[key]) # Calls Vector constructor passing an array. Question: this is only possible because the array is iterable?
        index = operator.index(key) # Better than int(key)
        return self._components[index]
    
    __match_args__ = ('x', 'y', 'z', 't') # Used for pattern matching and for dynamic access

    def __getattr__(self, name):
        cls = type(self)
        try:
            pos = cls.__match_args__.index(name)
        except ValueError:
            pos = -1
        if 0 <= pos < len(self._components):
            return self._components[pos]
        msg = f'{cls.__name__!r} object has no attribute {name!r}'
        raise AttributeError(msg)

    # Avoid trying to override any component directly
    def __setattr__(self, name: str, value: Any) -> None:
        cls = type(self)
        error = ''
        if len(name) == 1: # 1 letter special treatment
            if name in cls.__match_args__:
                error = f'readonly attribute {name!r}'
            elif name.islower():
                error = f"can't set attributes 'a' to 'z' in {cls.__name__!r}"
        if error:
            raise AttributeError(error)
        super().__setattr__(name, value) # default case
        

        pass

    @classmethod
    def frombytes(cls, octets):
        typecode = chr(octets[0])
        memv = memoryview(octets[1:]).cast(typecode)
        return cls(memv)

In [12]:
# Fourth version
# Makes vector hashable and improve == operator

from array import array
import functools
import operator
import reprlib
import math
from typing import Any

class Vector:
    typecode = 'd'

    def __init__(self, components) -> None:
        self._components = array(self.typecode, components)

    def __iter__(self):
        return iter(self._components)

    def __repr__(self) -> str:
        components = reprlib.repr(self._components) # Shorten the array representation
        components = components[components.find('['):-1] # Remove array([d' and last )
        return f'Vector({components})'
    
    def __str__(self) -> str:
        return str(tuple(self)) # Why print the whole Vector?

    def __bytes__(self):
        return (bytes([ord(self.typecode)]) + bytes(self._components))

    def __abs__(self) -> float:
        return math.hypot(*self) # math.hypot support n-dimensional input
    
    def __bool__(self) -> bool:
        return bool(abs(self))

    def __len__(self) -> int:
        return len(self._components)
    
    def __getitem__(self, key):
        if isinstance(key, slice):
            cls = type(self) 
            return cls(self._components[key]) # Calls Vector constructor passing an array. Question: this is only possible because the array is iterable?
        index = operator.index(key) # Better than int(key)
        return self._components[index]
    
    __match_args__ = ('x', 'y', 'z', 't') # Used for pattern matching and for dynamic access

    def __getattr__(self, name):
        cls = type(self)
        try:
            pos = cls.__match_args__.index(name)
        except ValueError:
            pos = -1
        if 0 <= pos < len(self._components):
            return self._components[pos]
        msg = f'{cls.__name__!r} object has no attribute {name!r}'
        raise AttributeError(msg)

    # Avoid trying to override any component directly
    def __setattr__(self, name: str, value: Any) -> None:
        cls = type(self)
        error = ''
        if len(name) == 1: # 1 letter special treatment
            if name in cls.__match_args__:
                error = f'readonly attribute {name!r}'
            elif name.islower():
                error = f"can't set attributes 'a' to 'z' in {cls.__name__!r}"
        if error:
            raise AttributeError(error)
        super().__setattr__(name, value) # default case

    def __hash__(self) -> int:
        hashes = (hash(x) for x in self._components) # generator expression
        return functools.reduce(operator.xor, hashes, 0) # Use third argument (initializer) to prevent TypeErrors
        
    def __eq__(self, other: object) -> bool:
        if len(self) != len(other):
            return False
        for a, b in zip(self, other):
            if a != b:
                return False
        return True

    @classmethod
    def frombytes(cls, octets):
        typecode = chr(octets[0])
        memv = memoryview(octets[1:]).cast(typecode)
        return cls(memv)

# Hash strategy

We've used the aggregated XOR operation compute the hash of our vector. We've applied the reduce operation in the vector with the XOR operation in order to create a way to identify uniquely the object.

Question: I did not get why using XOR operator instead of NOT operator?

In [19]:
# Fifth version
# Changes the representation, in case the user might want to see it's vector in hyperespheric coordinates.

from array import array
import functools
import itertools
import operator
import reprlib
import math
from typing import Any

class Vector:
    typecode = 'd'

    def __init__(self, components) -> None:
        self._components = array(self.typecode, components)

    def __iter__(self):
        return iter(self._components)

    def __repr__(self) -> str:
        components = reprlib.repr(self._components) # Shorten the array representation
        components = components[components.find('['):-1] # Remove array([d' and last )
        return f'Vector({components})'
    
    def __str__(self) -> str:
        return str(tuple(self)) # Why print the whole Vector?

    def __bytes__(self):
        return (bytes([ord(self.typecode)]) + bytes(self._components))

    def __abs__(self) -> float:
        return math.hypot(*self) # math.hypot support n-dimensional input
    
    def __bool__(self) -> bool:
        return bool(abs(self))

    def __len__(self) -> int:
        return len(self._components)
    
    def __getitem__(self, key):
        if isinstance(key, slice):
            cls = type(self) 
            return cls(self._components[key]) # Calls Vector constructor passing an array. Question: this is only possible because the array is iterable?
        index = operator.index(key) # Better than int(key)
        return self._components[index]
    
    __match_args__ = ('x', 'y', 'z', 't') # Used for pattern matching and for dynamic access

    def __getattr__(self, name):
        cls = type(self)
        try:
            pos = cls.__match_args__.index(name)
        except ValueError:
            pos = -1
        if 0 <= pos < len(self._components):
            return self._components[pos]
        msg = f'{cls.__name__!r} object has no attribute {name!r}'
        raise AttributeError(msg)

    # Avoid trying to override any component directly
    def __setattr__(self, name: str, value: Any) -> None:
        cls = type(self)
        error = ''
        if len(name) == 1: # 1 letter special treatment
            if name in cls.__match_args__:
                error = f'readonly attribute {name!r}'
            elif name.islower():
                error = f"can't set attributes 'a' to 'z' in {cls.__name__!r}"
        if error:
            raise AttributeError(error)
        super().__setattr__(name, value) # default case

    def __hash__(self) -> int:
        hashes = (hash(x) for x in self._components) # generator expression
        return functools.reduce(operator.xor, hashes, 0) # Use third argument (initializer) to prevent TypeErrors
        
    def __eq__(self, other: object) -> bool:
        if len(self) != len(other):
            return False
        for a, b in zip(self, other):
            if a != b:
                return False
        return True

    def angle(self, n):
        r = math.hypot(*self[n:])
        a = math.atan2(r, self[n - 1])
        if (n == len(self) - 1) and (self[-1] < 0):
            return 2.0 * math.pi - a
        else:
            return a
        
    def angles(self):
        return (self.angle(n) for n in range(1, len(self)))
    
    def __format__(self, fmt_spec: str = '') -> str:
        if fmt_spec.endswith('h'):
            fmt_spec = fmt_spec[:-1]
            coords = itertools.chain([abs(self)], self.angles())
            outer_fmt = '<{}>'
        else:
            coords = self
            outer_fmt = '({})'
        components = (format(c, fmt_spec) for c in coords)
        components = (format(c, fmt_spec) for c in coords)
        return outer_fmt.format(', '.join(components))


    @classmethod
    def frombytes(cls, octets):
        typecode = chr(octets[0])
        memv = memoryview(octets[1:]).cast(typecode)
        return cls(memv)

format(Vector([1, 1]), '.3eh')

1.4142135623730951